# Artificial Neural Network

## Import libraries

In [49]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [50]:
tf.__version__

'2.13.0'

# Part 1 - Data Preprocessing

## Import the dataset

In [51]:
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:,   -1].values

In [52]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [53]:
print(y)

[1 0 1 ... 1 1 0]


## Encoding categorical data

### Label Encoding the "Gender" column

In [54]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

In [55]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


### One Hot Encoding the "Geography" column

In [56]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [57]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


## Splitting the dataset into the Training set and Test set

In [58]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=0)

## Feature Scaling

In [59]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
xTrain = sc.fit_transform(xTrain)
xTest = sc.transform(xTest)

# Part 2 - Building the ANN

## Initializing the ANN

In [60]:
ann = tf.keras.models.Sequential()

## Adding the input layer and the first hidden layer

In [61]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## Adding the second hidden layer

In [62]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## Adding the output layer

In [63]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the ANN

## Compiling the ANN

In [64]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Training the ANN on the Training set

In [65]:
ann.fit(xTrain, yTrain, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 949us/step - loss: 0.5464 - accuracy: 0.7959
Epoch 2/100
250/250 [==============================] - 0s 695us/step - loss: 0.4668 - accuracy: 0.7961
Epoch 3/100
250/250 [==============================] - 0s 753us/step - loss: 0.4454 - accuracy: 0.7972
Epoch 4/100
250/250 [==============================] - 0s 816us/step - loss: 0.4341 - accuracy: 0.8030
Epoch 5/100
250/250 [==============================] - 0s 816us/step - loss: 0.4241 - accuracy: 0.8066
Epoch 6/100
250/250 [==============================] - 0s 753us/step - loss: 0.4126 - accuracy: 0.8115
Epoch 7/100
250/250 [==============================] - 0s 753us/step - loss: 0.3994 - accuracy: 0.8214
Epoch 8/100
250/250 [==============================] - 0s 753us/step - loss: 0.3871 - accuracy: 0.8329
Epoch 9/100
250/250 [==============================] - 0s 753us/step - loss: 0.3776 - accuracy: 0.8370
Epoch 10/100
250/250 [==============================] - 0s 690us/step - l

# Part 4 - Making the predictions and evaluating the model

## Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [66]:
tt = [[600, 'France', 'Male', 40, 3, 60000, 2, 1, 1, 50000]]
print(tt[0][1])
tt[0][2] = le.transform([tt[0][2]])
tt = np.array(ct.transform(tt))
tt = sc.transform(tt)
print(ann.predict(tt) > 0.5)
print(ann.predict(tt))

France
1/1 [==============================] - 0s 63ms/step
[[False]]
1/1 [==============================] - 0s 16ms/step
[[0.02812536]]


In [67]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

1/1 [==============================] - 0s 16ms/step
[[False]]
1/1 [==============================] - 0s 16ms/step
[[0.02812536]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

## Predicting the Test set results

In [68]:
yPred = ann.predict(xTest)
yPred = (yPred > 0.5)
print(np.concatenate((yPred.reshape(len(yPred), 1), yTest.reshape(len(yTest), 1)), 1))

63/63 [==============================] - 0s 755us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Making the Confusion Matrix

In [69]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(yTest, yPred)
print(cm)
accuracy_score(yTest, yPred)

[[1530   65]
 [ 206  199]]


0.8645